In [1]:
#######################
### Import Packages ###
#######################

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import geopandas as gpd
import altair as alt

## A. Import Data
All the data can be obtained via [openDOSM](https://open.dosm.gov.my), a website created by the Department of Statistics, Malaysia in an effort to make public sector data more accessible. The website consolidates data from various government department and agencies, but unfortunately not all. For the purpose of this analysis, only data with district-level breakdown is used. The data can be accessed via this [link](https://open.dosm.gov.my/data-catalogue?geography=DISTRICT).

In [2]:
########################
### Helper Functions ###
########################

def import_openDOSM(url, date_col=None):
    """
    Imports data in parquet format from OpenDOSM and returns a pandas DataFrame.
    Arguments:
        url (str): Parquet URL from OpenDOSM.
        date_col (str, optional): Name of a column to be parsed as datetime. (Default: None)
    Returns:
        df (DataFrame): The dataset in Pandas DataFrame format.
    """
    # Import the data.
    df = pd.read_parquet(url)
    # Change to datetime format, if any.
    if date_col is not None:
        df[date_col] = pd.to_datetime(df[date_col])
    return df

### A.1. Import Crime Data

In [3]:
# Import crime dataset.
df_crime = import_openDOSM('https://storage.data.gov.my/publicsafety/crime_district.parquet', 'date')

# Clean district names for crimes dataset.
rename_district_crime = {'district': {'Arau': 'Perlis', 'Kangar': 'Perlis', 'Padang Besar': 'Perlis',
                                      'Brickfields': 'W.P. Kuala Lumpur', 'Cheras': 'W.P. Kuala Lumpur', 'Dang Wangi': 'W.P. Kuala Lumpur', 'Sentul': 'W.P. Kuala Lumpur', 'Wangsa Maju': 'W.P. Kuala Lumpur',
                                      'Ampang Jaya': 'Gombak', 'Hulu Selangor': 'Ulu Selangor', 'Kajang': 'Ulu Langat', 'Klang Selatan': 'Klang', 'Klang Utara': 'Klang',
                                      'Petaling Jaya': 'Petaling', 'Serdang': 'Petaling', 'Sg. Buloh': 'Petaling', 'Shah Alam': 'Petaling', 'Subang Jaya': 'Petaling','Sungai Buloh': 'Petaling',
                                      'Batu Gajah': 'Kinta', 'Gerik': 'Hulu Perak', 'Ipoh': 'Kinta', 'Pengkalan Hulu': 'Hulu Perak', 'Sungai Siput': 'Kuala Kangsar', 'Taiping': 'Larut Dan Matang',
                                      'Tanjong Malim': 'Muallim', 'Tapah': 'Batang Padang', 'Iskandar Puteri': 'Johor Bahru', 'Johor Bahru Selatan': 'Johor Bahru', 'Johor Bahru Utara': 'Johor Bahru',
                                      'Kulaijaya': 'Kulai', 'Ledang': 'Tangkak', 'Nusajaya': 'Johor Bahru', 'Seri Alam': 'Johor Bahru', 'Cameron Highland': 'Cameron Highlands', 'Kuala Lipis': 'Lipis',
                                      'Bandar Bharu': 'Bandar Baharu', 'Nilai': 'Seremban', 'Kota Kinabatangan': 'Kinabatangan', 'Kota Samarahan': 'Samarahan', 'Matu Daro': 'Matu', 'Meradong': 'Sarikei',
                                      'Padawan': 'Kuching'}}
df_crime = df_crime.replace(rename_district_crime)
df_crime = df_crime[df_crime['district']!='All'].copy()
df_crime.loc[df_crime['district']=='W.P. Putrajaya', 'state'] = 'W.P. Putrajaya'
df_crime.loc[df_crime['district']=='W.P. Labuan', 'state'] = 'W.P. Labuan'

# Crime: Aggregate
df_crime_agg = df_crime[df_crime['type']=='all'].drop(columns=['type']).reset_index(drop=True)
df_crime_agg = df_crime_agg.groupby(['date', 'state', 'district', 'category'])['crimes'].sum().reset_index()    # Sum crime values at district level, since the renaming exercise earlier resulted in duplicate district observations.
df_crime_agg = df_crime_agg.pivot(index=['date', 'state', 'district'], columns='category', values='crimes')     # Pivot to wide format.
df_crime_agg['total'] = df_crime_agg['assault'] + df_crime_agg['property']                                      # Add a total column.
df_crime_agg.columns = ["crime_" + x for x in df_crime_agg.columns.values]                                      # Rename column for easier identification.
#df_crime_mod = df_crime_mod.reset_index()

# Crime: By type
df_crime_type = df_crime[df_crime['type']!='all']
df_crime_type = df_crime_type.groupby(['date', 'state', 'district', 'category', 'type'])['crimes'].sum().reset_index()      # Sum crime values at district level, since the renaming exercise earlier resulted in duplicate district observations.
df_crime_type = df_crime_type.pivot(index=['date', 'state', 'district'], columns=['category', 'type'], values='crimes')     # Pivot to wide format. 
df_crime_type.columns = ["_".join(x) for x in df_crime_type.columns.values.reshape(-1)]                                     # Flatten hierarchical column names
df_crime_type.columns = ["crime_" + x for x in df_crime_type.columns.values]                                                # Rename column for easier identification.

# Combine the crime tables.
df_crime_mod = df_crime_agg.merge(df_crime_type, on=['date', 'state', 'district'])
df_crime_mod = df_crime_mod.reset_index(level=0)
print(df_crime_mod.date.unique(), "\n")
print("Shape:", df_crime_mod.shape, "\n")
df_crime_mod.head()

<DatetimeArray>
['2016-01-01 00:00:00', '2017-01-01 00:00:00', '2018-01-01 00:00:00',
 '2019-01-01 00:00:00', '2020-01-01 00:00:00', '2021-01-01 00:00:00',
 '2022-01-01 00:00:00', '2023-01-01 00:00:00']
Length: 8, dtype: datetime64[ns] 

Shape: (1072, 16) 



date  crime_assault  crime_property  crime_total  \
state district                                                             
Johor Batu Pahat  2016-01-01            160             621          781   
      Johor Bahru 2016-01-01           1971            6746         8717   
      Kluang      2016-01-01            156             526          682   
      Kota Tinggi 2016-01-01            109             456          565   
      Kulai       2016-01-01            156             617          773   

                   crime_assault_causing_injury  crime_assault_murder  \
state district                                                          
Johor Batu Pahat                             39                     6   
      Johor Bahru                           470                    33   
      Kluang                                 58                     4   
      Kota Tinggi                            11                     8   
      Kulai                                  47                     7   

                   crime_assault_rape  crime_assault_robbery_gang_armed  \
state district                                                            
Johor Batu Pahat                   29                                 1   
      Johor Bahru                 118                                 6   
      Kluang                       37                                 2   
      Kota Tinggi                  19                                 0   
      Kulai                         4                                 0   

                   crime_assault_robbery_gang_unarmed  \
state district                                          
Johor Batu Pahat                                   56   
      Johor Bahru                                1008   
      Kluang                                       47   
      Kota Tinggi                                  50   
      Kulai                                        76   

                   crime_assault_robbery_solo_armed  \
state district                                        
Johor Batu Pahat                                  1   
      Johor Bahru                                 2   
      Kluang                                      0   
      Kota Tinggi                                 0   
      Kulai                                       0   

                   crime_assault_robbery_solo_unarmed  \
state district                                          
Johor Batu Pahat                                   28   
      Johor Bahru                                 334   
      Kluang                                        8   
      Kota Tinggi                                  21   
      Kulai                                        22   

                   crime_property_break_in  crime_property_theft_other  \
state district                                                           
Johor Batu Pahat                       115                         192   
      Johor Bahru                      936                        1264   
      Kluang                            41                         147   
      Kota Tinggi                       43                          60   
      Kulai                            163                         166   

                   crime_property_theft_vehicle_lorry  \
state district                                          
Johor Batu Pahat                                   16   
      Johor Bahru                                 300   
      Kluang                                       11   
      Kota Tinggi                                   9   
      Kulai                                        39   

                   crime_property_theft_vehicle_motorcar  \
state district                                             
Johor Batu Pahat                                      41   
      Johor Bahru                                   1171   
      Kluang                                          35   
      Kota Tinggi                               

### A.2. Import Population Data

In [4]:
# Import population dataset.
df_pop = import_openDOSM('https://storage.dosm.gov.my/population/population_district.parquet', 'date')

# Clean district names for the DOSM datasets.
rename_district_DOSM = {'district': {'Sp Selatan': 'Seberang Perai Selatan',
                                     'Sp Tengah': 'Seberang Perai Tengah',
                                     'Sp Utara': 'Seberang Perai Utara',
                                     'Cameron Highland': 'Cameron Highlands',
                                     'Larut dan Matang': 'Larut Dan Matang'}}
df_pop = df_pop.replace(rename_district_DOSM)

# Data: Total Overall Population
df_pop_tot = df_pop[((df_pop['sex']=='both') | (df_pop['sex']=='overall')) & (df_pop['age']=='overall') & (df_pop['ethnicity']=='overall')]    # Get overall population at district level.
df_pop_tot = df_pop_tot.drop(columns=['age', 'sex', 'ethnicity'])                                               # Drop other variables.
df_pop_tot = df_pop_tot.set_index(['state', 'district'])                                                # Set multi-index variables
print(df_pop_tot.date.unique(), "\n")
print("Shape:", df_pop_tot.shape, "\n")
df_pop_tot.head()

<DatetimeArray>
['2020-01-01 00:00:00', '2021-01-01 00:00:00', '2022-01-01 00:00:00',
 '2023-01-01 00:00:00', '2024-01-01 00:00:00']
Length: 5, dtype: datetime64[ns] 

Shape: (800, 2) 



date  population
state district                          
Johor Batu Pahat  2020-01-01       495.3
      Johor Bahru 2020-01-01      1711.2
      Kluang      2020-01-01       323.8
      Kota Tinggi 2020-01-01       222.4
      Kulai       2020-01-01       329.5

## B. Create Crime Time-Series Dataset

### B.1. Calculate Crime per Capita
District-level population data is only available since 2020. Therefore, for years prior 2020, population as at 2020 is used to calculate crime per capita. Assuming population growth is always positive, this may underestimate the actual crime rate as pre-2020 population would have been smaller.

In [5]:
########################
### Helper Functions ###
########################
def getCrimePerCapita(df_c, df_p, year_c, year_p):
    """
    Calculates crime per capita.
    Arguments:
        df_c (DataFrame): DataFrame containing crime values.
        df_p (DataFrame): DataFrame containing population values.
        year_c (int): Year of interest for crime data.
        year_p (int): Year of interest for population data.
    Returns:
        t_df (DataFrame): DataFrame of crime values divided by population values.
    """
    t = df_c[df_c['date'].dt.year==year_c]
    t = t.merge(df_p[df_p['date'].dt.year==year_p].drop(columns='date'), on=['state', 'district'])
    t = t.reset_index()
    t_df = t.copy().drop(columns=['state', 'district', 'date', 'population'])
    t_df = t_df.div(t.population, axis=0)
    t_df = t.copy()[['state', 'district', 'date', 'population']].merge(t_df, left_index=True, right_index=True)

    return t_df

In [6]:
# Calculate crime per capita.
t2016_df = getCrimePerCapita(df_crime_mod, df_pop_tot, 2016, 2020)
t2017_df = getCrimePerCapita(df_crime_mod, df_pop_tot, 2017, 2020)
t2018_df = getCrimePerCapita(df_crime_mod, df_pop_tot, 2018, 2020)
t2019_df = getCrimePerCapita(df_crime_mod, df_pop_tot, 2019, 2020)
t2020_df = getCrimePerCapita(df_crime_mod, df_pop_tot, 2020, 2020)
t2021_df = getCrimePerCapita(df_crime_mod, df_pop_tot, 2021, 2021)
t2022_df = getCrimePerCapita(df_crime_mod, df_pop_tot, 2022, 2022)
t2023_df = getCrimePerCapita(df_crime_mod, df_pop_tot, 2023, 2023)

# Concatenate all time periods into one long table.
crimePerPop_long = pd.concat([t2016_df, t2017_df, t2018_df, t2019_df, t2020_df, t2021_df, t2022_df, t2023_df]).reset_index(drop=True)
crimePerPop_long['date'] = crimePerPop_long['date'].dt.year
print("Shape:", crimePerPop_long.shape)
crimePerPop_long.head()

Shape: (1072, 19)


,state,district,date,population,crime_assault,crime_property,crime_total,crime_assault_causing_injury,crime_assault_murder,crime_assault_rape,crime_assault_robbery_gang_armed,crime_assault_robbery_gang_unarmed,crime_assault_robbery_solo_armed,crime_assault_robbery_solo_unarmed,crime_property_break_in,crime_property_theft_other,crime_property_theft_vehicle_lorry,crime_property_theft_vehicle_motorcar,crime_property_theft_vehicle_motorcycle
0,Johor,Batu Pahat,2016,495.3,0.323037,1.253786,1.576822,0.078740,0.012114,0.058550,0.002019,0.113063,0.002019,0.056531,0.232183,0.387644,0.032304,0.082778,0.518877
1,Johor,Johor Bahru,2016,1711.2,1.151823,3.942263,5.094086,0.274661,0.019285,0.068957,0.003506,0.589060,0.001169,0.195185,0.546985,0.738663,0.175316,0.684315,1.796985
2,Johor,Kluang,2016,323.8,0.481779,1.624460,2.106238,0.179123,0.012353,0.114268,0.006177,0.145151,0.000000,0.024707,0.126621,0.453984,0.033972,0.108091,0.901791
3,Johor,Kota Tinggi,2016,222.4,0.490108,2.050360,2.540468,0.049460,0.035971,0.085432,0.000000,0.224820,0.000000,0.094424,0.193345,0.269784,0.040468,0.202338,1.344424
4,Johor,Kulai,2016,329.5,0.473445,1.872534,2.345979,0.142640,0.021244,0.012140,0.000000,0.230653,0.000000,0.066768,0.494689,0.503794,0.118361,0.176024,0.579666


### B.2. Data Quality Check

In [7]:
# Check for missing value in crime dataset.
crimePerPop_long.isna().sum()

state                                      0
district                                   0
date                                       0
population                                 0
crime_assault                              0
crime_property                             0
crime_total                                0
crime_assault_causing_injury               0
crime_assault_murder                       0
crime_assault_rape                         0
crime_assault_robbery_gang_armed           0
crime_assault_robbery_gang_unarmed         0
crime_assault_robbery_solo_armed           0
crime_assault_robbery_solo_unarmed         0
crime_property_break_in                    0
crime_property_theft_other                 0
crime_property_theft_vehicle_lorry         0
crime_property_theft_vehicle_motorcar      0
crime_property_theft_vehicle_motorcycle    0
dtype: int64

In [8]:
# Check for missing geographical values.
district_boundaries = gpd.read_file('data/district_boundaries.shp')
crimePerPop_long_geo = pd.merge(district_boundaries[['district', 'geometry']], crimePerPop_long, left_on='district', right_on='district', how='outer')

missingval = crimePerPop_long_geo.set_index('district').isna().groupby('district').sum().sum(axis=1).sort_values(ascending=False)
missingval = missingval[missingval>0]
missingval.name = 'missing'

missingval_geo = pd.merge(district_boundaries[['district', 'geometry']], missingval, left_on='district', right_index=True, how='outer')
missingval_geo['missing'] = missingval_geo['missing'].fillna(0)


alt.Chart(missingval_geo).mark_geoshape(strokeWidth=1,stroke='lightgray',strokeOpacity=0.2
).encode(
    color=alt.Color('missing:O'),
    tooltip=['district:N','missing:O']
).properties(
    width=500,
    height=500
)

/opt/anaconda3/envs/inm433_visual/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

### B.3. Save the dataset
Uncomment the part below to save the file to the working folder.

In [ ]:
# Save the dataset to the folder.
#crimePerPop_long.to_csv("data/0_crimePerPop_long.csv", index=False)